In [1]:
import sys
import os

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    # sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')


os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from django.core import serializers

### Parsing MDB data

In [2]:
from mdb_stammdata_parser import *

In [9]:
#pm.Person.objects.filter(information_source="MDB Stammdata").all().delete()

(14405,
 {'parliament.Interjection_persons': 0,
  'parliament.ListMembership': 0,
  'parliament.Seat': 7800,
  'parliament.ConstituencyVote1': 0,
  'parliament.Post': 0,
  'parliament.Person': 6605})

In [8]:
#pm.Utterance.objects.filter(speaker__active_country=cmodels.Country.objects.get(name="Germany").id).delete()

(8994,
 {'scoping.Note': 0,
  'scoping.DocOwnership': 0,
  'tmv_app.DocTopic': 0,
  'parliament.Utterance_search_matches': 0,
  'parliament.Paragraph_search_matches': 0,
  'parliament.Interjection_parties': 2626,
  'parliament.Interjection_persons': 916,
  'parliament.Interjection': 2586,
  'parliament.Paragraph': 2423,
  'parliament.Utterance': 443})

In [3]:
parse_mdb_data()

read data from ./mdbs/MDB_STAMMDATEN.XML
<Element 'DOCUMENT' at 0x7f2b1677ea98>
Done. 24 warnings.


### Adding positions

**Positions added manually to spreadsheet, then added to database here**

In [2]:
import pandas as pd 
pos_add = pd.read_csv('mdb_positions_added.csv')
pos_add

,surname,first_name,position added
0,Klein,Hans,Vizepräsident
1,Wimmer,Willy,Parl. Staatssekretär
2,Schmidt,Renate,Vizepräsidentin
3,Schäfer,Helmut,Staatsminister
4,Klein,Hans,Bundesminister für wirtschaftliche Zusammenarbeit
5,Schäfer,Helmut,Staatsminister im Auswärtigen Amt
6,Geiger,Michaela,Parl. Staatssekretär beim Bundesminister für w...
7,Genscher,Hans-Dietrich,Bundesminister Auswärtigen
8,Töpfer,Klaus,"Bundesminister für Umwelt, Naturschutz und Rea..."
9,Töpfer,Klaus,Bundesminister


In [4]:
# will continue to add same position if repeated! 
# 0 - 73 added (server)
from django.core.exceptions import MultipleObjectsReturned

for i in range(0, len(pos_add)):
    try:
        surname = pos_add.iloc[i]['surname']
        first_name = pos_add.iloc[i]['first_name']
        position = pos_add.iloc[i]['position added']
        person = pm.Person.objects.get(surname=surname, first_name=first_name)
        if not position in person.positions:
            person.positions.append(position)
            person.save()
    except MultipleObjectsReturned:
        print(surname, first_name)

Weber Karl


In [25]:
# add jäger as alt surname to richard jaeger 
jaeger = pm.Person.objects.get(surname='Jaeger', first_name='Richard')
jaeger.alt_surnames.append('Jäger')
jaeger.positions.append('Vizepräsident')
jaeger.save()

In [27]:
# adding positions as there is more than one karl weber 
weber = pm.Person.objects.filter(surname='Weber', first_name='Karl', in_parlperiod__contains=['4'])[0]
weber.positions.append('Berichterstatter')
weber.save()

In [28]:
# adding positions as there is more than one karl müller 
muller = pm.Person.objects.get(surname='Müller', first_name = 'Karl', in_parlperiod__contains=['1'])
muller.positions.append('Berichterstatter')
muller.save()

In [29]:
# adding positions as there is more than one heinrich müller 
hmuller = pm.Person.objects.get(surname='Müller', first_name = 'Heinrich', in_parlperiod__contains=['1'])
hmuller.positions.append('Berichterstatter')
hmuller.save()

In [30]:
# add gerstenmaler as alt surname to eugen gerstenmaier 
gerstenmaier = pm.Person.objects.get(surname='Gerstenmaier', first_name='Eugen')
gerstenmaier.alt_surnames.append('Gerstenmaler')
gerstenmaier.save()

### Adding new people

In [5]:
pos2_add = pd.read_csv('nonelect_positions_added.csv')
pos2_add

,surname,first_name,wp,position added,ortszusatz,title,party,source
0,Albrecht,Ernst,"11, 10, 9, 8",Ministerpräsident,NaN,Dr.,CDU,Manual
1,Franke,Klaus,10,Senator,Berlin,NaN,CDU,Manual
2,Vogel,Bernhard,10,Ministerpräsident,Rheinland-Pfalz,NaN,CDU,Manual
3,Rau,Johannes,"13, 12, 11, 10, 9, 8",Ministerpräsident,Nordrhein-Westfalen,Dr. h. c.,SPD,Manual
4,Becker,Kurt,9,"Staatssekretär, Chef Presseund Informationsamt...",NaN,NaN,NaN,Manual
5,Becker,Kurt,9,Staatssekretär,NaN,NaN,NaN,Manual
6,Becker,Kurt,9,Staatssekretär im Bundeskanzleramt,NaN,NaN,NaN,Manual
7,Schmid,Albert,"9, 8",Staatssekretär im Bundesministerium für Raumor...,NaN,Dr.,SPD,Manual
8,Schmid,Albert,"9, 8",Staatssekretär,NaN,NaN,SPD,Manual
9,Lange,Volker,9,Senator,Hamburg,NaN,SPD,Manual


In [6]:
# ernst albrecht (hamburg) already exists, but not the right person
# so won't work with get_or_create
person = pm.Person(surname='Albrecht', first_name='Ernst')
position = 'Ministerpräsident'
title = 'Dr.'
party = 'CDU'
wp = pos2_add.loc[0]['wp']
person.positions = [position]
person.title = title
person.information_source = 'Manual'
person.in_parlperiod = []
for i in wp.split(','): 
        person.in_parlperiod.append(i)
person.save()

In [7]:
person = pm.Person.objects.filter(surname='Albrecht', first_name='Ernst')[1]
person.information_source = 'Manual'
person.save()

In [8]:
person = pm.Person(surname='Franke', first_name='Klaus')
position = 'Senator'
ortszusatz = 'Berlin'
party = 'CDU'
wp = pos2_add.loc[1]['wp']
person.positions = [position]
person.ortszusatz = ortszusatz
person.information_source = 'Manual'
person.in_parlperiod = []
for i in wp.split(','): 
        person.in_parlperiod.append(i)
person.save()

In [9]:
from django.core.exceptions import MultipleObjectsReturned
for i in range(2, len(pos2_add)):
    surname = pos2_add.iloc[i]['surname']
    first_name = pos2_add.iloc[i]['first_name']
    position = pos2_add.iloc[i]['position added']
    title = pos2_add.iloc[i]['title']
    party = pos2_add.iloc[i]['party']
    ortszusatz = pos2_add.iloc[i]['ortszusatz']
    source = pos2_add.iloc[i]['source']
    wp = pos2_add.iloc[i]['wp']
    try:    
        person, created = pm.Person.objects.get_or_create(surname=surname, first_name=first_name)
        if created:
            person.in_parlperiod = []
            person.positions = []
            person.positions = [position]
            person.information_source = source
            for i in wp.split(','): 
                person.in_parlperiod.append(i)   
            if title:
                person.title = title
            if party:
                try:
                    party_obj = pm.Party.objects.get(alt_names__contains=[party])
                    person.party = party_obj
                except pm.Party.DoesNotExist:
                    print("! Warning: party could not be identified: {}".format(party))
            if ortszusatz:
                person.ortszusatz = ortszusatz     
        else:
            person.positions.append(position)
            for i in wp.split(','): 
                person.in_parlperiod.append(i)
        person.save()
    except MultipleObjectsReturned:
        print(surname, first_name)

! Warning: party could not be identified: nan
! Warning: party could not be identified: nan


In [9]:
for i in range(2, len(pos2_add)):
    surname = pos2_add.iloc[i]['surname']
    first_name = pos2_add.iloc[i]['first_name']
    source = pos2_add.iloc[i]['source']
    person = pm.Person.objects.get(surname=surname, first_name=first_name)
    person.information_source = source 
    person.save()

### Testing

In [192]:
pm.Person.objects.get(surname='Hartmann', first_name = 'Alfred').in_parlperiod

[3, 2, 1, 3, 2, 1, 3, 2, 1]

In [36]:
pm.Person.objects.filter(surname='Albrecht', first_name = 'Ernst')

<QuerySet [<Person: Dr. Ernst Albrecht>, <Person: Ernst Albrecht (Hamburg)>]>

In [100]:
pm.Party.objects.get(name='cducsu').alt_names

['CDU',
 'CSU',
 'CDU/CSU',
 'Fraktion der Christlich Demokratischen Union/Christlich - Sozialen Union',
 'Fraktion der CDU/CSU (Gast)',
 'CSUS',
 'DSU']

In [106]:
pm.Party.objects.filter(alt_names__contains=['CSU'])

<QuerySet [<Party: CDUCSU>]>

In [190]:
import re
PERSON_POSITION = ['Vizepräsident(in)?', 'Präsident(in)?',
                   'Alterspräsident(in)?', 'Bundeskanzler(in)?',
                   'Staatsminister(in)?\s*(im)?.*$', '(?<=,\s)Bundesminister(in)?\s*(für)?.*$',
                   '(Par[li]\s?\.\s?)?Staatssekretär(in)?\s*(beim)?.*$', '(?<=,\s)Berichterstatter(in)?', 'Abg.',
                    'Ministerpräsident(in)?\s.*$', 'Minister(in)?', 'Senator(in)?']
PERSON_POSITION = re.compile(u'(%s)' % '|'.join(PERSON_POSITION), re.U)
name = 'Hellwege, Ministerpräsident Landes Niedersachsen'
PERSON_POSITION.sub('', name).strip(' ,')

'Hellwege'

In [9]:
# add party names

party = pm.Party.objects.get(name='cducsu')
an = party.alt_names
print(an)
an.append('cducsu')
party.alt_names = an
party.save()
print(party.alt_names)

['CDU', 'CSU', 'CDU/CSU', 'Fraktion der Christlich Demokratischen Union/Christlich - Sozialen Union', 'Fraktion der CDU/CSU (Gast)', 'CSUS', 'DSU']
['CDU', 'CSU', 'CDU/CSU', 'Fraktion der Christlich Demokratischen Union/Christlich - Sozialen Union', 'Fraktion der CDU/CSU (Gast)', 'CSUS', 'DSU', 'cducsu']


In [23]:
docs = pm.Document.objects.filter(parlperiod__n=13, sitting__lte=85)
docs.count()

84

In [24]:
docs.delete()

(169050,
 {'parliament.Document_search_matches': 0,
  'scoping.Note': 0,
  'scoping.DocOwnership': 0,
  'parliament.Utterance_search_matches': 0,
  'parliament.Paragraph_search_matches': 0,
  'parliament.Interjection_parties': 33393,
  'parliament.Interjection_persons': 17313,
  'parliament.Interjection': 46746,
  'parliament.Paragraph': 53342,
  'parliament.Utterance': 18172,
  'parliament.Document': 84})

In [25]:
persons = pm.Person.objects.filter(id__gte=62046).order_by('id')
persons.count()
print(persons)

<QuerySet [<Person: Ja, Lippold (Offenbach)>, <Person: Bitte Lippold (Offenbach)>, <Person: Schoppe, Schoppe>, <Person: Bitte Fuchs>, <Person: Bitte Lippelt>, <Person: Bitte Fuhrmann>, <Person: Kann Steindor>, <Person: Bitte Scheel>, <Person: Bitte Struck>, <Person: Bitte, Poß>, <Person: Herr Michels>, <Person: Lengsfeld. Lengsfeld>, <Person: Bitte Duve>, <Person: Bitte, Wodarg>, <Person: Dr. Gerhardt Gerhardt>, <Person: Sie Wolf>, <Person: Dr. Karl Hornhues>, <Person: Das Irmer>, <Person: Wir Beck>, <Person: Das Poppe>, '...(remaining elements truncated)...']>


In [26]:
persons.delete()

(108,
 {'parliament.Interjection_persons': 0,
  'parliament.ListMembership': 0,
  'parliament.Seat': 0,
  'parliament.ConstituencyVote1': 0,
  'parliament.Post': 0,
  'parliament.Person': 108})